## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploratory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2025">The Repo</a>.  

Once you have updated your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://github.com/apps/databricks">Databricks App on Github</a> or by cloning the repo to your laptop and then uploading the final_project directory and its contents to your workspace using file imports.  Your choice.

<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches to triggering your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)
- [In class examples - Spark Structured Streaming Performance](https://dbc-f85bdc5b-07db.cloud.databricks.com/editor/notebooks/2638424645880316?o=1093580174577663)

### Be sure your project runs end to end when *Run all* is executued on this notebook! (7 points)

### This project is worth 25% of your final grade.
- DSCC-202 Students have 55 possible points on this project (see points above and the instructions below)
- DSCC-402 Students have 60 possible points on this project (one extra section to complete)

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    # Optimize the tables
    optimize_table(BRONZE_DELTA)
    optimize_table(SILVER_DELTA)
    optimize_table(GOLD_DELTA)
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here (2 points)
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
# ENTER YOUR CODE HERE
from pyspark.sql.functions import *
from datetime import datetime, timedelta
import time
import matplotlib.pyplot as plt
import pandas as pd
import mlflow


total_cores = sc.defaultParallelism
# Spark will group files into ~200 partitions
spark.conf.set("spark.sql.shuffle.partitions", total_cores * 50)
# Optimize file reading
spark.conf.set("spark.sql.files.maxPartitionBytes", "64m")  # 64MB partition size to avoid tiny partitions
spark.conf.set("spark.sql.files.openCostInBytes", "4194304") # 4MB file open cost to coalesce small files

print(f"Set shuffle partitions to: {spark.conf.get('spark.sql.shuffle.partitions')}")

In [0]:
# import random
# files = dbutils.fs.ls(TWEET_SOURCE_PATH)

# # Creating a .5% subset of the data
# file_list = [f.path for f in files]

# file_sample = random.sample(file_list, 500)

# TWEET_SUBSET = "dbfs:/tmp/mkimani2/tweet_subset/"

# dbutils.fs.mkdirs(TWEET_SUBSET)

# def copy_file(src_path):
#     target_path = src_path.replace(TWEET_SOURCE_PATH, TWEET_SUBSET)
#     dbutils.fs.cp(src_path, target_path)


# for file_path in file_sample:
#     copy_file(file_path)

# print(f"Copied {len(file_sample)} files to {TWEET_SUBSET}")
# TWEET_SUBSET = "dbfs:/tmp/mkimani2/tweet_subset/"

## 2.0 Define and execute utility functions (3 points)
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
# ENTER YOUR CODE HERE
files = dbutils.fs.ls(TWEET_SOURCE_PATH)
display(spark.read.json(f"{TWEET_SOURCE_PATH}/999.json"))
print(f"File count: {len(files)}")

## 3.0 Transform the Raw Data to Bronze Data using a stream  (8 points)
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using delta lake to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defined in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
# ENTER YOUR CODE HERE
bronze_schema = """
    date STRING,
    user STRING,
    text STRING,
    sentiment STRING,
    source_file STRING,
    processing_time TIMESTAMP
"""
# Initialize empty table
spark.createDataFrame([], bronze_schema) \
     .write.format("delta") \
     .mode("overwrite") \
     .save(BRONZE_DELTA)

bronze_stream = (spark
            .readStream
            .format("cloudFiles")
            .schema(bronze_schema)
            .option("cloudFiles.format", "json")
            .load(TWEET_SOURCE_PATH)
            .withColumn("source_file", input_file_name())
            .withColumn("processing_time", current_timestamp())
            .writeStream
            .format("delta")
            .outputMode("append")
            .option("checkpointLocation", BRONZE_CHECKPOINT)
            .trigger(processingTime='10 seconds')
            .option("mergeSchema", "true")
            .queryName("bronze_stream")
            .start(BRONZE_DELTA))


## 4.0 Transform the Bronze Data to Silver Data using a stream (5 points)
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
# ENTER YOUR CODE HERE
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY") 

silver_schema = """
    timestamp TIMESTAMP,
    mention STRING,
    cleaned_text STRING,
    sentiment STRING
"""

# Initialize empty table
spark.createDataFrame([], silver_schema) \
     .write.format("delta") \
     .mode("overwrite") \
     .save(SILVER_DELTA)

silver_stream = (spark
                 .readStream
                 .format("delta")
                 .load(BRONZE_DELTA)
                 .withColumn("timestamp", to_timestamp(col("date"), "EEE MMM dd HH:mm:ss z yyyy"))
                 .withColumn("mentions", expr(r"regexp_extract_all(text, '@(\\\S+)', 1)"))
                 .withColumn("mention", explode(col("mentions")))
                 .withColumn("cleaned_text", regexp_replace(col("text"), r"@\S+", ""))
                 .select("timestamp",
                         "mention",
                         "cleaned_text",
                         "sentiment")
                 .writeStream
                 .format("delta")
                 .outputMode("append")
                 .option("checkpointLocation", SILVER_CHECKPOINT)
                 .option("ignoreDeletes", True)
                 .trigger(processingTime='10 seconds')
                 .queryName("silver_stream")
                 .start(SILVER_DELTA))

## 5.0 Transform the Silver Data to Gold Data using a stream (7 points)
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
# ENTER YOUR CODE HERE
gold_schema = """
    timestamp TIMESTAMP,
    mention STRING, 
    cleaned_text STRING, 
    sentiment STRING, 
    predicted_score DOUBLE, 
    predicted_sentiment STRING, 
    sentiment_id INTEGER, 
    predicted_sentiment_id INTEGER
"""
result_schema = """
predicted_score DOUBLE,
predicted_sentiment STRING
"""

@pandas_udf(result_schema, functionType=PandasUDFType.SCALAR_ITER)
def predict_sentiment(tweets):
    model = mlflow.pyfunc.load_model(f"models:/{MODEL_NAME}/Production")
    for tweet in tweets:
        input_data = {'text': tweet}
        predictions = model.predict(input_data)
        print(predictions)
        yield pd.DataFrame({
            "predicted_score": predictions["score"],
            "predicted_sentiment": predictions["label"]
        })
# Initialize empty table
spark.createDataFrame([], gold_schema) \
     .write.format("delta") \
     .mode("overwrite") \
     .save(GOLD_DELTA)

gold_stream = (spark
               .readStream
               .format("delta")
               .load(SILVER_DELTA)
               .withColumn("sentiment_id", when(col("sentiment") == "positive", lit(1)).otherwise(lit(0)))
               .withColumn("predictions", predict_sentiment(col("cleaned_text")))
               .withColumn("predicted_score", col("predictions.predicted_score"))
               .withColumn("predicted_sentiment", col("predictions.predicted_sentiment"))
               .withColumn("predicted_sentiment_id", 
                           when(col("predicted_sentiment") == "POS", lit(1))
                           .when(col("predicted_sentiment") == "NEG", lit(0))
                           .otherwise(lit(-1)))
               .drop("predictions")
               .writeStream
               .format("delta")
               .outputMode("append")
               .option("checkpointLocation", GOLD_CHECKPOINT)
               .trigger(processingTime='10 seconds')
               .queryName("gold_stream")
               .start(GOLD_DELTA)
)

## 6.0 Monitor your Streams (5 points)
- Setup a loop that runs at least every 10 seconds
- Print a timestamp of the monitoring query along with the list of streams, rows processed on each, and the processing time on each
- Run the loop until all of the data is processed (0 rows read on each active stream)
- Plot a line graph that shows the data processed by each stream over time
- Plot a line graph that shows the average processing time on each stream over time

In [0]:
# ENTER YOUR CODE HERE
from datetime import datetime
import matplotlib.pyplot as plt
import time

monitoring_data = []
while True:
    timestamp = datetime.now()
    active_streams = spark.streams.active

    print(f"\n=== Metrics at {timestamp} ===")
    exit_flags = [0, 0, 0]
    stream_metrics = {}
    for stream in active_streams:
        
        progress = stream.lastProgress
        if progress:
            rows = progress['numInputRows']
            name = stream.name
            processing_time = progress['durationMs'].get('triggerExecution', 0)

            metrics = {"name": name,
                       "rows": rows,
                       "processing_time": processing_time,
                       "timestamp": timestamp}
            stream_metrics[name] = metrics

            print(f"Stream: {name}")
            print(f"Rows processed: {rows}")
            print(f"Processing time: {processing_time}")

            if (stream.name == "bronze_stream" and rows == 0):
                exit_flags[0] = 1
            elif (stream.name == "silver_stream" and rows == 0):
                exit_flags[1] = 1
            elif (stream.name == "gold_stream" and rows == 0):
                exit_flags[2] = 1
    
    monitoring_data.append(stream_metrics)

    sum = 0
    for flag in exit_flags:
        sum += flag
    if sum == 3:
        break
    
    time.sleep(10)

# Viz
# Prepare data structures
stream_stats = {}

# Process monitoring data
for snapshot in monitoring_data:
    for name, metrics in snapshot.items():
        if name not in stream_stats:
            stream_stats[name] = {
                "timestamps": [],
                "rows": [],
                "processing_times": []
            }
        stream_stats[name]["timestamps"].append(metrics["timestamp"])
        stream_stats[name]["rows"].append(metrics["rows"])
        stream_stats[name]["processing_times"].append(metrics["processing_time"])

# Create plots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Plot rows processed
for name, stats in stream_stats.items():
    ax1.plot(stats["timestamps"], stats["rows"], label=name, marker='o')
ax1.set_title("Rows Processed Over Time")
ax1.set_ylabel("Rows")
ax1.legend()
ax1.grid(True)

# Plot processing time
for name, stats in stream_stats.items():
    ax2.plot(stats["timestamps"], stats["processing_times"], label=name, marker='x')
ax2.set_title("Processing Time Over Time")
ax2.set_ylabel("Time (ms)")
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

## 7.0 Bronze Data Exploratory Data Analysis (5 points)
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
# ENTER YOUR CODE HERE
from pyspark.sql.functions import sum
time.sleep(10)
bronze_df = (spark
             .read
             .load(BRONZE_DELTA))
print(f"Bronze table tweet count: {bronze_df.count()}")
null_counts = bronze_df.agg(*[sum(isnull(c).cast("int")).alias(c + " null count")
                              for c in bronze_df.columns])
display(null_counts)

tweets_per_user = (bronze_df
                   .groupBy(col("user"))
                   .agg(
                       count(col("text")).alias("tweet_count")
                   )
                   .sort("tweet_count", ascending=False))
display(tweets_per_user)

mention_count = (bronze_df
                 .agg(
                     sum((col("text").contains('@').cast("int"))).alias("gte_1_mention")
                 )
                 .withColumn("no_mentions", bronze_df.count() - col("gte_1_mention")))
display(mention_count)

display(tweets_per_user.limit(20))


Databricks visualization. Run in Databricks to view.

## 8.0 Capture the accuracy metrics from the gold table in MLflow  (4 points)
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the model name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
# ENTER YOUR CODE HERE
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

time.sleep(10)
gold_df = spark.read.load(GOLD_DELTA)
y_true = gold_df.select(col('sentiment_id')).toPandas()['sentiment_id']
y_pred = gold_df.select(col('predicted_sentiment_id')).toPandas()['predicted_sentiment_id']
with mlflow.start_run():
    # Calculate metrics
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    # Log metrics
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    # Create and save confusion matrix plot
    cm = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots()
    ax.matshow(cm, cmap=plt.cm.Blues)
    for (i, j), v in np.ndenumerate(cm):
        ax.text(j, i, v, ha='center', va='center')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.savefig("confusion_matrix.png")
    # Log matrix as artifact
    mlflow.log_artifact("confusion_matrix.png")
    # Log parameters
    mlflow.log_param("model_name", MODEL_NAME)
    mlflow.log_param("mlflow_version", mlflow.__version__)
    # Get & log SILVER_DELTA
    delta_version = (spark.sql(f"DESCRIBE HISTORY '{SILVER_DELTA}'")
                     .agg({"version": "max"})
                     .collect()[0][0])
    mlflow.log_param("delta_tbl_version", delta_version)


## 9.0 Application Data Processing and Visualization (6 points)
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
# ENTER YOUR CODE HERE
application_schema = """
    min_timestamp TIMESTAMP, 
    max_timestamp TIMESTAMP, 
    mention STRING, 
    negative LONG, 
    neutral LONG, 
    positive LONG, 
    total LONG
"""
time.sleep(10)
application_df = (spark.read
                  .load(GOLD_DELTA)
                  .groupBy("mention")
                  .agg(
                      min("timestamp").alias("min_timestamp"),
                      max("timestamp").alias("max_timestamp"),
                      sum(when(col("sentiment") == 'negative', 1).otherwise(0)).alias("negative"),
                      sum(when(col("sentiment") == 'neutral', 1).otherwise(0)).alias("neutral"),
                      sum(when(col("sentiment") == 'positive', 1).otherwise(0)).alias("positive"),
                      sum(col("sentiment")).alias("total")
                  )
                  .sort("total", ascending=False))

print(f"Total mentions: {application_df.count()}")

positive_mentions = application_df.select('mention', 'positive').sort("positive", ascending=False).limit(20)
negative_mentions = application_df.select('mention', 'negative').sort("negative", ascending=False).limit(20)
display(positive_mentions)
display(negative_mentions)

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

## 10.0 Clean up and completion of your pipeline (3 points)
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook. Note: In the includes there is a variable START_TIME that captures the starting time of the notebook.

In [0]:
# ENTER YOUR CODE HERE
from datetime import timedelta

for stream in spark.streams.active:
    print(f"{stream.name} is active.")
    stream.stop()
    print(f"{stream.name} has been stopped.")

print(f"Elapsed time: {timedelta(seconds = time.time() - START_TIME)}")


## 11.0 How Optimized is your Spark Application (Grad Students Only) (5 points)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### ENTER YOUR MARKDOWN HERE